In [1]:
#Menjalankan fast API dengan mode loop sehingga bisa menservise terus
import nest_asyncio
nest_asyncio.apply()

In [ ]:
pip install openpyxl

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
import os
import uvicorn

app = FastAPI()

# Nama file Excel
file_path = "customers.xlsx"

# Fungsi untuk membaca data dari file Excel ke dalam dictionary
def load_customers_from_excel():
    if os.path.exists(file_path):
        df = pd.read_excel(file_path, engine='openpyxl', index_col="ID")
        return df.to_dict(orient='index')
    else:
        # Jika file tidak ada, kembalikan dictionary kosong
        return {}

# Fungsi untuk menyimpan data ke file Excel
def save_customers_to_excel(customers):
    df = pd.DataFrame.from_dict(customers, orient='index')
    df.index.name = "ID"  # Set index column name
    df.to_excel(file_path, engine='openpyxl')

# Load customer data dari file Excel saat server dijalankan
customers_db = load_customers_from_excel()

# Model Pydantic untuk customer
class Customer(BaseModel):
    name: str
    address: str
    phone: str
    age: int

# Endpoint untuk menambahkan data customer baru
@app.post("/customers/")
def create_customer(customer: Customer):
    new_id = max(customers_db.keys(), default=0) + 1  # Default ke 0 jika kosong
    customers_db[new_id] = customer.dict()
    
    # Simpan data ke Excel
    save_customers_to_excel(customers_db)
    
    return {"id": new_id, "customer": customer}

# Endpoint untuk mendapatkan semua customer
@app.get("/customers/")
def get_customers():
    return customers_db

# Endpoint untuk mendapatkan customer tertentu berdasarkan ID
@app.get("/customers/{customer_id}")
def read_customer(customer_id: int):
    customer = customers_db.get(customer_id)
    if customer is None:
        raise HTTPException(status_code=404, detail="Customer not found")
    return customer

# Endpoint untuk memperbarui data customer
@app.put("/customers/{customer_id}")
def update_customer(customer_id: int, customer: Customer):
    if customer_id not in customers_db:
        raise HTTPException(status_code=404, detail="Customer not found")
    
    customers_db[customer_id] = customer.dict()
    
    # Simpan perubahan ke Excel
    save_customers_to_excel(customers_db)
    
    return {"id": customer_id, "customer": customer}

# Endpoint untuk menghapus customer
@app.delete("/customers/{customer_id}")
def delete_customer(customer_id: int):
    if customer_id not in customers_db:
        raise HTTPException(status_code=404, detail="Customer not found")
    
    del customers_db[customer_id]
    
    # Simpan perubahan ke Excel
    save_customers_to_excel(customers_db)
    
    return {"message": f"Customer {customer_id} deleted successfully"}

# Jalankan server FastAPI menggunakan Uvicorn
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [5016]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57784 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57812 - "GET /customers/ HTTP/1.1" 200 OK


In [ ]:
#program dari bapanya

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import openpyxl

# Inisialisasi aplikasi FastAPI
app = FastAPI()

# Nama file Excel
EXCEL_FILE = "products.xlsx"

# Model Pydantic untuk validasi input
class Product(BaseModel):
    name: str
    price: float
    description: str = None

# Fungsi untuk membaca data produk dari Excel
def read_products_from_excel():
    workbook = openpyxl.load_workbook(EXCEL_FILE)
    sheet = workbook.active
    products = {}
    for row in sheet.iter_rows(min_row=2, values_only=True):  # Lewati header
        product_id, name, price, description = row
        products[product_id] = {"name": name, "price": price, "description": description}
    return products

# Fungsi untuk menulis data produk ke Excel
def write_products_to_excel(products):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(["ID", "Name", "Price", "Description"])  # Header
    for product_id, product_data in products.items():
        sheet.append([product_id, product_data["name"], product_data["price"], product_data.get("description", "")])
    workbook.save(EXCEL_FILE)

# Inisialisasi database dari Excel
try:
    fake_products_db = read_products_from_excel()
except FileNotFoundError:
    # Jika file Excel tidak ditemukan, buat file baru dengan data awal
    fake_products_db = {
        1: {"name": "Laptop", "price": 15000, "description": "Laptop gaming high-end"},
        2: {"name": "Smartphone", "price": 7000, "description": "Smartphone terbaru dengan kamera bagus"},
    }
    write_products_to_excel(fake_products_db)

# Endpoint untuk menambahkan produk baru
@app.post("/products/")
def create_product(product: Product):
    new_id = max(fake_products_db.keys()) + 1
    fake_products_db[new_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan ke Excel
    return {"id": new_id, "product": product}

# Endpoint untuk menampilkan semua produk
@app.get("/products/")
def get_products():
    return fake_products_db

# Endpoint untuk menampilkan produk berdasarkan ID
@app.get("/products/{product_id}")
def read_product(product_id: int):
    product = fake_products_db.get(product_id)
    if product is None:
        raise HTTPException(status_code=404, detail="Product not found")
    return product

# Endpoint untuk memperbarui produk berdasarkan ID
@app.put("/products/{product_id}")
def update_product(product_id: int, product: Product):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    fake_products_db[product_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"id": product_id, "product": product}

# Endpoint untuk menghapus produk berdasarkan ID
@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    del fake_products_db[product_id]
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"message": f"Product {product_id} deleted successfully"}

# Jalankan server FastAPI menggunakan Uvicorn
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [11172]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
